# Creating an end-to-end automated trading system

What are we going to build?
---------------------------

A day trading system for trading in National Stock Exchange(NSE).

Orders would be placed at the start of the day and closed by the end of the day or exit by a stop loss. We would be using the end of the day data as our source data. 

The trading system has four distinct parts

1. Creating a trading strategy
2. Backtesting and simulation
3. Deploying the system and order execution
4. Evaluating the performance of the system


We follow a simple principle
> **Don't waste your money**

# Primer on financial concepts

A short primer on risk and return calculations. This is more of a practical explanation; so take it with a grain of salt.
 

## Putting it simply

> ### **Risk** is the amount of money you are willing to lose

> ### **Return** is the amount of money you expect to gain

A risk to return ratio of 1:2 mean you are willing to lose 1 rupee if you expect a gain of 2 rupees.

The risk-reward ratio is often misleading since

 * there are no probabilities attached to the risk and return
 * we strictly expect either we lose 1 or gain 2 but in practice we may end up between these two cases most of the time
 
So, we would be using **payoff** to evaluate strategies




Payoff is the amount of money you would expect to receive for the amount of investment made.

It is the average returns and losses multiplied by the probability of returns and losses. Let's look at an example

Let's assume a trading strategy with a risk-reward ratio of 1:2 with the following details

 * lose 1 - 30% of the time
 * gain 2 - 20% of the time
 * makes 0 - 50% of the time
 
Then the pay off is 

`-1*0.3 + 2*0.2 + 0*0.5 = 0.1`

So, you would making 0.1 for every rupee invested.

Payoff makes decision making easier. A positive payoff is favorable while a negative payoff is unfavorable

**Warning**

We expect these probabilities in the long run based on history. So, we would very well end up in loss in the short or medium term even if our probabilities turn out to be right.

### Rule 01

> ### Risk is very very personal

### Rule 02

> ### Returns are always expectations unless realized

### Return calculations

Return calculation is similar to net profit calculation

$Profit = Sell \ Value - Buy \ Value$

$Returns = \frac{Sell Value - Buy Value}{Buy Value}$

In case of annualized returns, then

$Annualized Returns = \frac{Sell Value - Buy Value}{Buy Value} * \frac{365}{holding\ period}$

 * Holding period is the period you held the investment
 * Buy and sell value is the quantity multiplied by average price
 
Warning
=========
 > Annualized return is not the actual return (unless the holding period is exactly 365 days). We are just proportionately calculating the returns based on the holding period. So if the holding period is 2 months, we assume we would get these returns for the rest of the year, which is quite impractical. Thus we must distinguish properly between holding period returns and annualized returns.


### Rule 03

> ### Know your assumptions and always write them down

**Example**

100 units of stock A is bought on 1st JAN 2019 for 2500 and sold on 25th MAR 2019 for 2620. Calculate the returns

In [12]:
import pandas as pd

# Inputs
buy_price = 2500
buy_date = pd.to_datetime('2019-01-01')
sell_price = 2620
sell_date = pd.to_datetime('2019-03-25')
qty = 100

# Calculations
buy_value = buy_price * qty
sell_value = sell_price * qty
holding_period = (sell_date - buy_date).days # Using pandas attribute to get the days

# Outputs
profit = sell_value - buy_value
returns = (sell_value - buy_value)/buy_value
annualized_return = returns * (365/holding_period)

print('Net profit  = {}'.format(profit))
print('Returns for the period = {:.2f}%'.format(returns*100))
print('Annualized returns = {:.2f}%'.format(annualized_return*100))

Net profit  = 12000
Returns for the period = 4.80%
Annualized returns = 21.11%


In [14]:
# Exercise 01 - Calculate the returns
# 1000 units of an index bought for 11000
# and sold for 12000 after holding for 60 days


In [ ]:
# Exercise 02 - Calculate the returns
# 100 units of a stock bought on 1st JAN 2018 for 3400
# and sold for 3185 on 20th JULY 2019